# Retrieving Product Information using AWS Product Advertising API - Part 2
Now that the AWS Product Advertising API responses have been saved for each queried product(ASIN), the next step is to parse this information. The following fields will be needed for advertising purposes:
1. ASIN
2. Title
3. Brand
4. GTIN(s)
5. MPN/Model
6. Specifications/Details
7. Product Description
8. Features
9. Product Category
10. Stock Image(s)

Information from the AWS Product Advertising API responses will be used to fill as many fields as possible.

## Import Libraries, Packages, and Modules

In [1]:
# import libraries, packages, and modules
import pandas as pd # Data structures and data analysis tools library
from bs4 import BeautifulSoup # HTML and XML document parsing package
from re import search, sub

## Construct pandas DataFrame from Tab-Seperated Values File of AWS Product Advertising API Responses by ASIN

In [2]:
# construct pandas DataFrame with data from tsv file of aws pa api responses by asin
column_number_with_ASIN = 0 # set this variable to the numberical value of the csv column containing the desired ASINs
asin_responses_df = pd.read_csv('aws_pa_item_attributes_responses.tsv', header=0, sep='\t')
print(asin_responses_df.shape) # print DataFrame Shape
asin_responses_df.head() # preview DataFrame

(107, 5)


,asin,item_attributes_xml,editorial_review_xml,images_xml,date
0,B004XC6GJ0,"<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...",2019-01-29
1,B000P1DEHU,"<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...",2019-01-29
2,B004NEUJKA,"<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...",2019-01-29
3,B0011ULQNI,"<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...",2019-01-29
4,B001O0DP48,"<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...","<?xml version=""1.0"" ?><html><body><itemlookupr...",2019-01-29


## Construct pandas DataFrame for the Desired Product Fields

In [3]:
aws_pa_product_data_df = pd.DataFrame(columns=['asin', 'title', 'brand', 'gtin', 'mpn_model', 'specs', 'description', 'features', 'category', 'image'])
aws_pa_product_data_df.head() # return dataframe head

,asin,title,brand,gtin,mpn_model,specs,description,features,category,image


## Define Functions for Parsing XML Data

In [4]:
# define function for parsing xml data to prevent script from breaking on error, returns blank string instead
def parse_xml(xml, elementstring):
    """
    Parses product attributes from xml data.
    """
    xml = BeautifulSoup(xml, 'lxml')
    try:
        if elementstring == 'image':
            return(xml.item.largeimage.url.text)
        elif elementstring == 'description':
            try:
                desc_string = xml.find_all('content')[0].get_text()
                return(desc_string[0:search('<', str(image_test.find_all('content')[0].text)).span()[0]])
            except:
                return(xml.find_all('content')[0].get_text())
        elif elementstring == 'features':
            feature_list = []
            for feature in xml.find_all('feature'):
                if 'warranty' in feature.text.lower():
                    pass
                elif 'report to amazon' in feature.text.lower():
                    pass
                else:
                    feature_list.append(feature.text)
            if len(feature_list) > 0:
                return(feature_list)
            else:
                return('')
        elif elementstring == 'mpn':
            model_mpn1 = str(xml.find_all('model')[0].text)
            model_mpn2 = str(xml.find_all('mpn')[0].text)
            if model_mpn1 != '':
                return(model_mpn1)
            else:
                return(model_mpn2)
        elif 'list' in elementstring:
            list_elements = []
            for list_item in xml.find(elementstring).findAll(elementstring + 'element', recursive=False):
                list_elements.append(list_item.text)
            return(list_elements)
        elif len(xml.find_all(elementstring)) > 1:
            list_elements = []
            for element in xml.find_all(elementstring):
                list_elements.append(element.text)
            return(list_elements)
        else:
            return(str(xml.find_all(elementstring)[0].text))
    except:
        return('')

In [5]:
# define function for parsing specifications from xml data to prevent script from breaking on error, returns blank string instead
def spec_parse_xml(xml):
    """
    Parses product specifications from xml data.
    """
    try:
        xml = BeautifulSoup(xml, 'lxml')
        field_spec_list = []
        specs_map_dict = {}
        for element in xml.itemattributes.children:
            if 'sku' in element.name.lower() or 'warranty' in element.name.lower() or 'feature' in element.name.lower() or 'legaldisclaimer' in element.name.lower() or 'language' in element.name.lower() or 'department' in element.name.lower() or 'dimensions' in element.name.lower() or 'title' in element.name.lower() or 'product' in element.name.lower():
                    pass
            elif 'listprice' in element.name.lower():
                pass
            elif 'list' in element.name.lower():
                specs_map_dict[element.name] = 'list'
            elif len(xml.find_all(element.name)) > 1:
                specs_map_dict[element.name] = 'multi'
            else:
                specs_map_dict[element.name] = 'single'
        for attrib in specs_map_dict:
            specs_value_list = []
            if specs_map_dict[attrib] == 'list':
                if attrib == 'catalognumberlist':
                    for cn in xml.catalognumberlist.children:
                        specs_value_list.append(cn.text)
                    specs_map_dict[attrib] = list(set(specs_value_list))
                elif attrib == 'eanlist':
                    for cn in xml.eanlist.children:
                        specs_value_list.append(cn.text)
                    specs_map_dict[attrib] = list(set(specs_value_list))
                elif attrib == 'upclist':
                    for cn in xml.eanlist.children:
                        specs_value_list.append(cn.text)
                    specs_map_dict[attrib] = list(set(specs_value_list))
            elif specs_map_dict[attrib] == 'multi':
                for value in xml.find_all(attrib):
                    specs_value_list.append(value.text)
                specs_map_dict[attrib] = list(set(specs_value_list))
            elif specs_map_dict[attrib] == 'single':
                specs_map_dict[attrib] = xml.find_all(attrib)[0].text
            else:
                pass
        for key in specs_map_dict:
            dict_string = ''
            dict_string += key + ': '
            if type(specs_map_dict[key]) == list:
                    for i, list_items in enumerate(specs_map_dict[key]):
                        if i < len(specs_map_dict[key]) - 1:
                            dict_string += list_items + ', '
                        else:
                            dict_string += list_items
            else:
                dict_string += specs_map_dict[key]
            field_spec_list.append(dict_string)
        return(field_spec_list)
    except:
        return('')

In [6]:
# define function for construction product category taxonomy
def cat_taxonomy(productgroup, producttype):
    """
    Constructs product taxonomy from provided arguments
    """
    cat_string = '{} > {}'.format(productgroup, producttype)
    if cat_string == ' > ':
        return('')
    else:
        return(cat_string)

In [7]:
# construct product class
class product:
    """
    Class constructs product object and defines the objects product attributes
    """
    def __init__(self, item_attributes_xml, editorial_review_xml, images_xml):
        self.ia_xml = item_attributes_xml
        self.i_xml = images_xml
        self.er_xml = editorial_review_xml
        self.image = parse_xml(self.i_xml, 'image')
        self.description = parse_xml(self.er_xml, 'description')
        self.asin = parse_xml(self.ia_xml, 'asin')
        self.title = parse_xml(self.ia_xml, 'title')
        self.brand = parse_xml(self.ia_xml, 'brand')
        self.upc = parse_xml(self.ia_xml, 'upc')
        self.ean = parse_xml(self.ia_xml, 'ean')
        self.isbn = parse_xml(self.ia_xml, 'isbn')
        self.mpn = parse_xml(self.ia_xml, 'mpn')
        self.model = parse_xml(self.ia_xml, 'model')
        self.features = parse_xml(self.ia_xml, 'features')
        self.department = parse_xml(self.ia_xml, 'department')
        self.group = parse_xml(self.ia_xml, 'productgroup')
        self.type = parse_xml(self.ia_xml, 'producttypename')
        self.specs = spec_parse_xml(self.ia_xml)

In [8]:
# function for iterating through each product
def parse_loop(df, column_names):
    """
    1. Takes DataFrame of product xml data.
    2. Iterates through rows creating an object of the product class.
    3. Constructs a dictionary from the object attributes.
    4. Returns a DataFrame of desired product fields.
    """
    new_df = pd.DataFrame(columns=column_names)
    for row in df.iterrows():
        catalog_dict = {}
        catalog_item = product(row[1][1], row[1][2], row[1][3])
        catalog_dict = {
            'asin' : row[1][0],
            'title' : catalog_item.title,
            'brand' : catalog_item.brand,
            'gtin' : catalog_item.ean,
            'mpn_model' : catalog_item.mpn,
            'specs' : catalog_item.specs,
            'description' : catalog_item.description,
            'features' : catalog_item.features,
            'category' : cat_taxonomy(catalog_item.group, catalog_item.type),
            'image' : catalog_item.image,
        }
        new_df = new_df.append(catalog_dict, ignore_index=True)
    return(new_df)

## Construct DataFrame of Product Data

In [9]:
# Construct DataFrame of Product Data 
aws_pa_product_data_df = parse_loop(asin_responses_df, aws_pa_product_data_df.columns)
aws_pa_product_data_df

,asin,title,brand,gtin,mpn_model,specs,description,features,category,image
0,B004XC6GJ0,ARRIS SURFboard SB6121 4x4 DOCSIS 3.0 Cable Mo...,ARRIS,0168141495601,SB6121,"[binding: Personal Computers, brand: ARRIS, ca...",A new and improved version of the world's most...,"[Compatible with Time Warner Cable, Charter, C...",CE > COMPUTER_INPUT_DEVICE,https://images-na.ssl-images-amazon.com/images...
1,B000P1DEHU,Sony DVD/VCR Progressive Scan Combo Player SLV...,Sony,0027242708952,LYSB000P1DEHU-ELECTRNCS,"[binding: Electronics, brand: Sony, ean: 00272...",Sony's SLV-D281P simplifies your home theater ...,[Combination DVD Player and Hi-Fi VCR - Compon...,CE > VCR,https://images-na.ssl-images-amazon.com/images...
2,B004NEUJKA,Uniden D1660-2 DECT6.0 Caller ID Cordless hand...,Uniden,0999993504940,D1660-2,"[binding: Office Product, brand: Uniden, catal...",You'll love that this Uniden Caller ID/ Call W...,"[Digital DECT 6.0 for superior clarity, securi...",CE > PHONE,https://images-na.ssl-images-amazon.com/images...
3,B0011ULQNI,,,,,,,,,
4,B001O0DP48,Uniden DECT 6.0 Silver Cordless Phone with Cal...,Uniden,0050633271667,DECT1560-2,"[binding: Office Product, brand: Uniden, catal...",Advanced DECT 6.0 technology operates on the n...,[DECT 6.0 Interference Free Cordless Frequency...,CE > PHONE,https://images-na.ssl-images-amazon.com/images...
5,B00DQV2BDO,Panasonic KX-TGA402N 1.9GHz Extra Handset,Panasonic,0037988482528,,"[binding: Office Product, brand: Panasonic, ea...","This auction is for the handset only. Charger,...",[DECT 6.0 Plus Technology - Interference Free...,CE > OFFICE_ELECTRONICS,https://images-na.ssl-images-amazon.com/images...
6,B000E1B2SO,,,,,,,,,
7,B00005K3B1,Uniden EXA2950 900 MHz Cordless Phone with Dig...,Uniden,0050633151624,EXA-2950,"[binding: Office Product, brand: Uniden, catal...","For a busy home or office, consider the Uniden...",[900 MHz cordless phone with digital answering...,CE > PHONE,https://images-na.ssl-images-amazon.com/images...
8,B00000J0E6,RCA VHS Rewinder (Discontinued by Manufacturer),RCA,0079000309598,Video Rewinder -Machine,"[binding: Electronics, brand: RCA, catalognumb...",RCA UVR1Q Features: Quickly Rewinds VHS Video ...,[1-Way VHS Rewinder],Speakers > BLANK_MEDIA,https://images-na.ssl-images-amazon.com/images...
9,B0001B86GI,Uniden DCX640 2.4 GHz Accessory Handset for DC...,Uniden,0050633320129,DCX640,"[binding: Office Product, brand: Uniden, catal...",2.4GHz Accessory Handset - 2.4GHz Accessory ha...,[2.4 GHz digital signal; includes charger base...,CE > PHONE,https://images-na.ssl-images-amazon.com/images...


## Save aws_pa_product_data_df state in a Tab-delimited values file
As this process may be repeated over time and existing data may remain useful, the mode is set to 'a' for append.

In [10]:
# save as tsv
aws_pa_product_data_df.to_csv('aws_pa_product_data.tsv', sep='\t' , index=False, mode='a')
print('Export complete.')

Export complete.


## Construct List of Products Missing the Title Field

In [11]:
# Construct List of Products Missing the Title Field
aws_pa_product_data_missing_df = aws_pa_product_data_df['asin'][aws_pa_product_data_df['title'] == '']
aws_pa_product_data_missing_df

3     B0011ULQNI
6     B000E1B2SO
26    B0057OCS9A
27    B0015F0A8G
42    B00006IIOL
44    B0007Z70YM
46    B000Q8UPKQ
Name: asin, dtype: object

## Results

In [12]:
# Number of Products with Title Present
total = aws_pa_product_data_df.shape[0]
title_present = aws_pa_product_data_df['asin'][aws_pa_product_data_df['title'] != ''].shape[0]
title_absent = aws_pa_product_data_df['asin'][aws_pa_product_data_df['title'] == ''].shape[0]
print('Number of Product Rows with Title Present: ' + str(title_present))
print('Number of Product Rows with Title Absent: ' + str(title_absent))
print('Total Number of Product Rows: ' + str(total))
print('Success Rate: ' + str(round(title_present / total * 100, 2)) + '%')

Number of Product Rows with Title Present: 100
Number of Product Rows with Title Absent: 7
Total Number of Product Rows: 107
Success Rate: 93.46%


## Discussion

This was the second part of a two part project. The goal of parsing the desired product fields was largely successful. As depicted in the <i>aws_pa_product_data_missing_df</i> DataFrame, data about some products is unavailable for retrieval via the Amazon Web Services Product Advertising API. Consequently, data for these products will need to be obtained from a different source. One alternative source that may be useful is eBay. A future project may address these shortcomings in the interest of improving data quality and completeness. This project was successful with at  93.46% of products now being matched with at least a product title.